In [10]:
import os
from Bio import SeqIO
import io
from Bio.Seq import Seq
import subprocess

In [11]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [12]:
def make_raw_files_for_alignment(gene_sequence,annotated_genome_location,annotated_species_name,error_exon,query_fasta_sequence,query_length):
    for offset in range(3):
        translated_sequence = str(gene_sequence[offset:].translate()).split("*")
        for i in range(len(translated_sequence)):
            if len(translated_sequence[i])> 0.8*query_length:
                sequence_set = f">set{i+1}_frame{offset}\n{translated_sequence[i]}\n\n"
                # print(i+1, offset)
                
                with open(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}/for_alignment/{error_exon}_translated_genomic_sequence_{i+1}_frame{offset}.fa",'w') as out_file:
                    output = f"{query_fasta_sequence}\n\n{sequence_set}"
                    out_file.write(output)

In [13]:
def run_mafft(annotated_genome_location,annotated_species_name,error_exon):
    location = f'{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}/for_alignment'
    list_of_files_to_run_mafft_on = os.listdir(location)
    for file in list_of_files_to_run_mafft_on:
        if file.endswith(".fa"):
            command = f'"mafft" --localpair --maxiterate 16 --reorder --distout "{location}/{file}" > "{location}/alignment_{file}.txt"'
            # print(command)
            subprocess.run(f'{command}', shell=True, stderr = subprocess.DEVNULL) 
            # os.system(f'{command}')
            command = f'"mafft" --localpair --clustalout --maxiterate 16 --reorder "{location}/{file}" > "{location}/alignment_clustal_{file}.txt"'
            subprocess.run(f'{command}', shell=True, stderr = subprocess.DEVNULL) 
            
            # os.system(f'{command}')
    return(location)

In [14]:
def process_mafft_output(mafft_run_folder, error_exon):
    list_of_files_in_mafft_run_folder = os.listdir(mafft_run_folder)
    score_output = []
    score = 99
    min_score_sequence = ''
    alignment_file = ''
    for file in list_of_files_in_mafft_run_folder:
        if file.endswith(".fa.hat2"):
            with io.open(f"{mafft_run_folder}/{file}", 'r') as dist_matrix_file:
                dist_matrix_list = dist_matrix_file.readlines()
            
            sequence_name = dist_matrix_list[-2].rstrip().split("=")[1]
            distance_score = float(dist_matrix_list[-1].rstrip())
            if len(score_output) < 5:
                score_output.append( [sequence_name,distance_score])
            else:
                for i in range(len(score_output)):
                    score_at_this_index = score_output[i][1]
                    if distance_score < score_at_this_index:
                        score_output[i] = [sequence_name,distance_score]
                        break
            if distance_score < score:
                score = distance_score
                min_score_sequence = sequence_name
                alignment_file = f'alignment_{file.replace(".hat2",".txt")}'
    # print(score_output)
    print(f"min = {min_score_sequence}, {score}" )
    # score_out_merged = '\n'.join(score_output)
    print(f"5 top scores:\n{score_output}")
    print(f"{mafft_run_folder}/{alignment_file}")
    
    alignment_file = SeqIO.parse(f"{mafft_run_folder}/{alignment_file}", 'fasta')
    # print (records.id)
    fasta_start_position = 0
    fasta_end_position = 0
    start_switch = 0
    end_switch = 0
    alignment_name = ''
    for records in alignment_file:
        
        
        
        if error_exon in records.id:
            gap_counter = 0
            base_counter = 0
            for current_position in range(len(records.seq)):
                sequence_length = len(records.seq) - records.seq.count('-')
                # print(f"fasta_end_position {fasta_end_position} fasta_start_position {fasta_start_position}")
                # print(f"current_position = {current_position}, {len(records.seq)}")
                # print(records.seq[current_position])
                # print("fasta_start_position",fasta_start_position)
                # print("start_switch",start_switch)
                
                # print(gap_counter, base_counter)
                # print(5,0.2*sequence_length)
                if start_switch == 1 and records.seq[current_position] == "-" and gap_counter > 3 and (len(records.seq[:current_position]) - records.seq[:current_position].count('-')) < (0.1*sequence_length):
                #  :
                    # print("\n\nhere\n\n")
                    start_switch = 0
                    gap_counter = 0

                if "-" not in records.seq[current_position] and start_switch == 0 :
                    fasta_start_position = current_position
                    start_switch = 1
                
                    
                
                    
                if end_switch == 1 and "-" not in records.seq[current_position] and (len(records.seq[current_position:]) - records.seq[current_position:].count('-')) > 0.1*sequence_length :
                    end_switch = 0
                    gap_counter = 0
                
                    
                if start_switch == 1 and records.seq[current_position] == "-" and end_switch == 0:
                    # print(f"base_counter {base_counter}")
                    # print(f"fasta_end_position {fasta_end_position}")
                    
                    # print(f"fasta_end_position {fasta_end_position}")
                    
                    fasta_end_position = current_position
                    end_switch = 1
                if "-" in records.seq[current_position]:
                    gap_counter += 1
                else:
                    gap_counter = 0
                    base_counter += 1
        else:
            alignment_name = records.id
            if fasta_end_position == 0:
                
                fasta_end_position = current_position
            end_switch = 1
            start_switch = 1

        if start_switch == 1 and end_switch == 1:
            print(fasta_start_position, fasta_end_position)
            gene_sequence = records.seq[fasta_start_position:fasta_end_position]
            print(f"{records.id}\n{gene_sequence}")
    return(gene_sequence, alignment_name)

# process_mafft_output(mafft_run_folder, error_exon)

In [15]:
def get_genome_file(genome_location,species):
    list_of_files_in_genome_folder = os.listdir(f"{genome_location}/{species}")
    for file in list_of_files_in_genome_folder:
        if file.endswith("_genomic.fna"):
            genome_file = file
    return(genome_file)  

def get_annotated_genome_name(annotated_genome_location, species):
    list_of_annotated_genomes = os.listdir(annotated_genome_location)
  
    # print(annotated_species)
    for annotated_species in list_of_annotated_genomes:
        if annotated_species.endswith(species):
            return(annotated_species)
             
    if annotated_species_name == '':
        print(f"Error with annotated species name")
        assert False

def check_and_make_folders(annotated_genome_location,annotated_species_name,error_exon):
    list_of_files_inside_annotated_species_folder = os.listdir(f"{annotated_genome_location}/{annotated_species_name}")

    if f"Period_gene_genomic_sequence_individual_exon_{query_species}" not in list_of_files_inside_annotated_species_folder:
        os.mkdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}")
    
    list_of_exon_directories = os.listdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/")
    
    if error_exon not in list_of_exon_directories:
        os.mkdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}")
        os.mkdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}/for_alignment")
    elif error_exon in list_of_exon_directories:
        list_of_folders_1 = os.listdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}")
        if "for_alignment" not in list_of_folders_1:
            os.mkdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}/for_alignment")
        list_of_files = os.listdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}/for_alignment")
        for file in list_of_files:
            os.remove(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}/for_alignment/{file}")


In [16]:
def get_gene_sequence(genome_location, species, genome_file, scaffold,gene_start,gene_end,complement,annotated_genome_location,annotated_species_name):
    from Bio import SeqIO
    print("Getting Gene")
    list_of_files_inside_annotated_species_folder = os.listdir(f"{annotated_genome_location}/{annotated_species_name}")
    # print(list_of_files_inside_annotated_species_folder)
    # print(
    if f"Period_gene_genomic_sequence_individual_exon_{query_species}" not in list_of_files_inside_annotated_species_folder:
        os.mkdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}")

    list_of_files_inside_indiv_exon_folder = os.listdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}")
    if "temp" not in list_of_files_inside_indiv_exon_folder:
         os.mkdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/temp")

    # os.system(f'samtools faidx "{genome_location}/{species}/{genome_file}"')
    # subprocess.run("pwd")
    subprocess.run(f'samtools faidx "{genome_location}/{species}/{genome_file}"', shell = True, stderr = subprocess.DEVNULL)
    # os.system(f'samtools faidx "{genome_location}/{species}/{genome_file}" {scaffold}:{gene_start}-{gene_end} > "{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/temp/temp_genome.fa"')
    subprocess.run(f'samtools faidx "{genome_location}/{species}/{genome_file}" {scaffold}:{gene_start}-{gene_end} > "{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/temp/temp_genome.fa"', shell = True, stderr = subprocess.DEVNULL)
    genome = SeqIO.parse(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/temp/temp_genome.fa", "fasta")
    for entries in genome:
        gene_sequence = entries.seq
        if complement == "1":
            gene_sequence = gene_sequence.reverse_complement()
        break
    # print(gene_sequence)
    return (gene_sequence)

In [17]:
def mafft_process(previous_exon_coordinates,
                  next_exon_coordinates,
                  current_exon_coordinates,
                  query_species,
                  query_transcript,
                  query_location,
                  annotated_genome_location,
                 genome_location):
    query_exon = current_exon_coordinates[6].split("query")[-1][1:]
    error_exon = query_exon
    # print("AA")
    genome_file = get_genome_file(genome_location,species)
    
    upstream_exon_line, downstream_exon_line = previous_exon_coordinates.split(","),next_exon_coordinates.split(",")
    complement,scaffold = upstream_exon_line[4], upstream_exon_line[1]

    if complement == "0":
        gene_start = min(int(upstream_exon_line[2]),int(upstream_exon_line[3]))
        gene_end = max(int(downstream_exon_line[2]),int(downstream_exon_line[3]))
    
    if complement == "1":
        gene_start = min(int(downstream_exon_line[2]),int(downstream_exon_line[3]))
        gene_end = max(int(upstream_exon_line[2]),int(upstream_exon_line[3]))
    
    
    with open(f"{query_location}/{query_species}/{query_transcript}/query_{query_exon}.fa", 'r') as query_file:
        query_file_list = query_file.readlines()
        query_fasta_sequence = "".join(query_file_list)
        left_overhang = query_file_list[0].split("Frame")[1][1]
        right_overhang = query_file_list[0].split("rightoh")[1][1]
        original_query_name = query_file_list[0]
        original_query = query_file_list[1]
        
    query_length = len(query_fasta_sequence.split("\n")[1])
    
    annotated_species_name = get_annotated_genome_name(annotated_genome_location, species)
    # print(annotated_species_name)

    # print(annotated_genome_location)
    gene_sequence = get_gene_sequence(genome_location, species, genome_file, scaffold,gene_start,gene_end,complement,annotated_genome_location,annotated_species_name)
    # assert False
    check_and_make_folders(annotated_genome_location,annotated_species_name,error_exon)
    
    
    make_raw_files_for_alignment(gene_sequence,annotated_genome_location,annotated_species_name,error_exon,query_fasta_sequence,query_length)
    
    return(annotated_species_name, error_exon,species, genome_file,gene_sequence,left_overhang,right_overhang,scaffold, original_query_name,original_query )


In [19]:

blast_output_location = "/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/8.Heliconiinae_Danainae/1.Blast_result"
# blast_output_location = "/mnt/j/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/3.Satyrine/1.Blast_result"
species_list = os.listdir(blast_output_location)
if "desktop.ini" in species_list:
    species_list.remove("desktop.ini")

species_list.remove("Hestina_assimilis")
species_list.remove("Charaxes_varanes")
for species in species_list:
    annotated_genome_location = "/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/8.Heliconiinae_Danainae/1.Blast_result"
    query_location = "/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/8.Heliconiinae_Danainae/1.Query"
    list_of_query_species = os.listdir(query_location)
    
    if "desktop.ini" in list_of_query_species:
        list_of_query_species.remove("desktop.ini")
    list_of_query_species = ["20.Vanessa_tameamea"]
    for query_species in list_of_query_species:
        list_of_query_transcripts = os.listdir(f"{query_location}/{query_species}")
        list_of_query_transcripts.remove("desktop.ini")
        for query_transcript in list_of_query_transcripts:


            genome_location = "/mnt/f/Genomes_2023-12-26"



            output_coordinate_file = "Species," + "Scaffold," + "Start," + "Stop," + "Complement," + "Error," + "Gene,"+ "Query_start," + "Query_stop,"+ "Query_Length," +  "AG_GT," + "Spliceator_prediction\n"
            exons_to_check_list = ["Exon_2","Exon_3","Exon_4","Exon_5","Exon_6","Exon_7","Exon_8","Exon_9","Exon_10","Exon_11","Exon_12","Exon_13","Exon_14","Exon_15","Exon_16","Exon_17","Exon_18","Exon_19","Exon_20","Exon_21","Exon_22","Exon_23","Exon_24","Exon_25"]
#             exons_to_check_list = ["Exon_21"]

            list_of_files_in_species_folder = os.listdir(f"{blast_output_location}/{species}")
            coordinate_file_name = ''
        #     print(list_of_files_in_species_folder)
            for file_names in list_of_files_in_species_folder:

                if file_names.endswith("_coordinates_old.csv"):
                    coordinate_file_name = file_names
            if coordinate_file_name =='':
                print(f"Coordinate file error")
                assert False

            with io.open(f"{blast_output_location}/{species}/{coordinate_file_name}", 'r') as temp_file_open:
                coordinate_file_lines = temp_file_open.readlines()
            # print(coordinate_file_lines[0])
            for exons_to_check in exons_to_check_list:

                for i in range(1,len(coordinate_file_lines)):
                    coordinate_file_lines_split = coordinate_file_lines[i].split(",")
                    current_exon_coordinates = coordinate_file_lines[i].split(",")
                    if i == 1:
                        scaffold_first_exon = coordinate_file_lines[i].split(",")[1]
                        first_exon_start, first_exon_end = coordinate_file_lines[i].split(",")[2],coordinate_file_lines[i].split(",")[3]
                    # if coordinate_file_lines_split[5] == "Y":
                    if  coordinate_file_lines_split[6].endswith(exons_to_check):
                        if i == 1:
                            print(f'First Exon has errors\n{coordinate_file_lines[i]}')
                            assert False
                        else:
                            print(f"Scaffold = {scaffold_first_exon}, start = {first_exon_start}, end = {first_exon_end}")
                            print(coordinate_file_lines[i])
                            # process_current_exon = input("Error Found! Process?")
                            process_current_exon = 'y'
                            if process_current_exon.lower()[0] == "y":
                                if i>2:
                                    previous_exon_number = i-2
                                else:
                                    previous_exon_number = i-1
                                if i+2 <=int(exons_to_check_list[-1].split("_")[1]):
                                    next_exon_number = i+2
                                else:
                                    next_exon_number = i+1
                                if i >= int(exons_to_check_list[-1].split("_")[1]):
                                    next_exon_number = int(exons_to_check_list[-1].split("_")[1])
#                                     continue

                                '''
                                the above code ensures that missing exons do not cause issues
                                '''

                                previous_exon_coordinates = ''
                                next_exon_coordinates = ''
                                while True:
                                    print(previous_exon_number)
                                    if coordinate_file_lines[previous_exon_number].split(",")[5] == "N":
                                        previous_exon_coordinates = coordinate_file_lines[previous_exon_number]
                                        break
                                    # else:
                                    #     proceed_prompt = input(f"Previous exon coordinate\n{coordinate_file_lines[previous_exon_number]}\nProceed?")
                                    #     if proceed_prompt.lower()[0] == "y":
                                    #         previous_exon_coordinates = coordinate_file_lines[previous_exon_number]
                                    #         break
                                    previous_exon_number -= 1
                                while True:
                                    print(next_exon_number)
                                    try:
                                        if coordinate_file_lines[next_exon_number].split(",")[5] == "N":
                                            next_exon_coordinates = coordinate_file_lines[next_exon_number]
                                            break
                                    except:
                                        if coordinate_file_lines[next_exon_number-1].split(",")[5] == "N":
                                            next_exon_coordinates = coordinate_file_lines[next_exon_number-1]
                                            break
                                   
                                    next_exon_number += 1


                                print(previous_exon_coordinates)
                                print(next_exon_coordinates)
                                annotated_species_name, error_exon, species, genome_file, gene_sequence,left_overhang,right_overhang,scaffold,original_query_name, original_query = mafft_process(previous_exon_coordinates,
                                                                                                                                  next_exon_coordinates,
                                                                                                                                  current_exon_coordinates,
                                                                                                                                  query_species,
                                                                                                                                  query_transcript,
                                                                                                                                  query_location,
                                                                                                                                  annotated_genome_location,
                                                                                                                                     genome_location)

                                mafft_run_folder = run_mafft(annotated_genome_location,annotated_species_name,error_exon)

                                possible_gene_sequence, alignment_name = process_mafft_output(mafft_run_folder, error_exon)

                                print(f"Original query = {original_query}")
                                print(f"New query = {possible_gene_sequence}")
                                
                                list_of_folders_inside_exon_folder = os.listdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}")
                                if "for_blast" not in list_of_folders_inside_exon_folder:
                                    os.mkdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}/for_blast")
                                else:
                                    list_of_files = os.listdir(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}/for_blast")
                                    for file in list_of_files:
                                        os.remove(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}/for_blast/{file}")


                                with open(f"{annotated_genome_location}/{annotated_species_name}/Period_gene_genomic_sequence_individual_exon_{query_species}/{error_exon}/for_blast/new_query.txt",'w') as query_file:
                                    query = f">Query_{error_exon}_{alignment_name}\n{possible_gene_sequence}"
                                    query_file.write(query)
                    

Scaffold = CAKASE010000043.1, start = 6369470, end = 6369558
Danaus_chrysippus,CAKASE010000043.1,6369627,6369710,0,N,13.Danaus_plexippus_XM_061526465.1_Frame_1_rightoh_2_query_Exon_2,1,27,27

1
4
Danaus_chrysippus,CAKASE010000043.1,6369470,6369558,0,N,13.Danaus_plexippus_XM_061526465.1_Frame_0_rightoh_2_query_Exon_1,1,29,29

Danaus_chrysippus,CAKASE010000043.1,6370670,6370764,0,N,13.Danaus_plexippus_XM_061526465.1_Frame_2_rightoh_0_query_Exon_4,1,31,31

Getting Gene
min = set4_frame2, 0.204
5 top scores:
[['set4_frame2', 0.204], ['set38_frame1', 1.664], ['set8_frame1', 1.719], ['set10_frame0', 1.798], ['set16_frame0', 1.774]]
/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/8.Heliconiinae_Danainae/1.Blast_result/Danaus_chrysippus/Period_gene_genomic_sequence_individual_exon_20.Vanessa_tameamea/Exon_2/for_alignment/alignment_Exon_2_translated_genomic_sequence_4_frame2.fa.txt
29 56
20.Vanessa_tameamea_XM_026643498.1_Frame_1_rightoh_2_query_Exon_2
YSSKSMHSGSNSSASSGYGG

Getting Gene
min = set46_frame1, 0.283
5 top scores:
[['set19_frame1', 1.867], ['set37_frame1', 1.825], ['set46_frame1', 0.283]]
/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/8.Heliconiinae_Danainae/1.Blast_result/Danaus_chrysippus/Period_gene_genomic_sequence_individual_exon_20.Vanessa_tameamea/Exon_9/for_alignment/alignment_Exon_9_translated_genomic_sequence_46_frame1.fa.txt
7 82
20.Vanessa_tameamea_XM_026643498.1_Frame_2_rightoh_2_query_Exon_9
ANEIIVNAKPFVIRHSATGSLDYIDTESVPYLGYLPQDIVQKDALQLYHPSDLGYLRQVYEAIVKEGVVERSKPY
7 82
set46_frame1
SAEIIHNAIPFVIRHSATGSLEYIDHESVPYLGYLPQDIVEKDALQLYHPGDLGYLRQIYETIVKEGGVQRSKPY
Original query = ANEIIVNAKPFVIRHSATGSLDYIDTESVPYLGYLPQDIVQKDALQLYHPSDLGYLRQVYEAIVKEGVVERSKPY
New query = SAEIIHNAIPFVIRHSATGSLEYIDHESVPYLGYLPQDIVEKDALQLYHPGDLGYLRQIYETIVKEGGVQRSKPY
Scaffold = CAKASE010000043.1, start = 6369470, end = 6369558
Danaus_chrysippus,CAKASE010000043.1,6374739,6374850,0,N,13.Danaus_plexippus_XM_061526465.1_Frame_1_rightoh_0_quer

min = set28_frame1, 0.963
5 top scores:
[['set38_frame0', 1.855], ['set28_frame1', 0.963], ['set33_frame1', 1.781], ['set14_frame2', 1.872]]
/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/8.Heliconiinae_Danainae/1.Blast_result/Danaus_chrysippus/Period_gene_genomic_sequence_individual_exon_20.Vanessa_tameamea/Exon_16/for_alignment/alignment_Exon_16_translated_genomic_sequence_28_frame1.fa.txt
28 85
20.Vanessa_tameamea_XM_026643498.1_Frame_1_rightoh_0_query_Exon_16
HNSEMEKDLVKIHRDSRIYSKVEKEKISDEMKQKKREHLARCNAYIQQSILENSEQK
28 85
set28_frame1
HNAEMERELIKMYRENR-SSKGDREKASNETRQKKKQHLARCNAAFQPTSLGLSDPQ
Original query = HNSEMEKDLVKIHRDSRIYSKVEKEKISDEMKQKKREHLARCNAYIQQSILENSEQK
New query = HNAEMERELIKMYRENR-SSKGDREKASNETRQKKKQHLARCNAAFQPTSLGLSDPQ
Scaffold = CAKASE010000043.1, start = 6369470, end = 6369558
Danaus_chrysippus,CAKASE010000043.1,6379020,6379164,0,N,13.Danaus_plexippus_XM_061526465.1_Frame_0_rightoh_1_query_Exon_17,4,48,48

15
19
Danaus_chrysippus,CAKASE010000

Getting Gene
min = set49_frame0, 1.578
5 top scores:
[['set63_frame1', 1.758], ['set67_frame1', 1.773], ['set75_frame2', 1.785], ['set36_frame1', 1.807], ['set49_frame0', 1.578]]
/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/8.Heliconiinae_Danainae/1.Blast_result/Danaus_chrysippus/Period_gene_genomic_sequence_individual_exon_20.Vanessa_tameamea/Exon_23/for_alignment/alignment_Exon_23_translated_genomic_sequence_49_frame0.fa.txt
2 44
20.Vanessa_tameamea_XM_026643498.1_Frame_0_rightoh_0_query_Exon_23
CWRKPNQCGDAMPFQ-SYEMVNSEIKDKIAPRRNIEPPWMKQ
2 44
set49_frame0
SWHKNQKGDDLMSLQSSTEAVTYTPNKGQVQQKKVDPSWVEE
Original query = CWRKPNQCGDAMPFQSYEMVNSEIKDKIAPRRNIEPPWMKQ
New query = SWHKNQKGDDLMSLQSSTEAVTYTPNKGQVQQKKVDPSWVEE
Scaffold = CAKASE010000043.1, start = 6369470, end = 6369558
Danaus_chrysippus,CAKASE010000043.1,6383278,6383379,0,N,13.Danaus_plexippus_XM_061526465.1_Frame_0_rightoh_0_query_Exon_24,1,34,34

22
25
Danaus_chrysippus,CAKASE010000043.1,6382357,6382515,0,N

Getting Gene
min = set16_frame2, 0.114
5 top scores:
[['set16_frame2', 0.114], ['set3_frame2', 1.487], ['set59_frame2', 1.588], ['set69_frame2', 1.591], ['set39_frame2', 1.651]]
/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/8.Heliconiinae_Danainae/1.Blast_result/Danaus_plexippus/Period_gene_genomic_sequence_individual_exon_20.Vanessa_tameamea/Exon_7/for_alignment/alignment_Exon_7_translated_genomic_sequence_16_frame2.fa.txt
18 38
20.Vanessa_tameamea_XM_026643498.1_Frame_0_rightoh_1_query_Exon_7
DRNTFASQITSGLTIPKNVN
18 38
set16_frame2
DRNTFASQITSGLAVPKNVN
Original query = DRNTFASQITSGLTIPKNVN
New query = DRNTFASQITSGLAVPKNVN
Scaffold = NC_083559.1, start = 6597896, end = 6597984
Danaus_plexippus,NC_083559.1,6602103,6602306,0,N,13.Danaus_plexippus_XM_061526465.1_Frame_2_rightoh_1_query_Exon_8,1,67,67

6
10
Danaus_plexippus,NC_083559.1,6599466,6599597,0,N,13.Danaus_plexippus_XM_061526465.1_Frame_0_rightoh_0_query_Exon_6,1,44,44

Danaus_plexippus,NC_083559.1,6602941

...

min = set42_frame2, 0.249
5 top scores:
[['set42_frame2', 0.249], ['set197_frame1', 1.693], ['set230_frame0', 1.729], ['set221_frame1', 1.734], ['set249_frame0', 1.75]]
/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/8.Heliconiinae_Danainae/1.Blast_result/Vanessa_cardui/Period_gene_genomic_sequence_individual_exon_20.Vanessa_tameamea/Exon_17/for_alignment/alignment_Exon_17_translated_genomic_sequence_42_frame2.fa.txt
22 67
set42_frame2
AQGVKRSSKQLEEGTAHKHRCSSARLMRQKQIARAANTQTTPTMG
Original query = AHGVKRSSKQLEEGTAHKHRCSSARLIRQKQIARAANTQSTPTPTM
New query = AQGVKRSSKQLEEGTAHKHRCSSARLMRQKQIARAANTQTTPTMG
Scaffold = NC_061154.1, start = 11468772, end = 11468905
Vanessa_cardui,NC_061128.1,6121754,6121858,0,Y,13.Danaus_plexippus_XM_061526465.1_Frame_2_rightoh_1_query_Exon_18,3,33,35

16
15
14
20
21
22
23
24
Vanessa_cardui,NC_061154.1,11458563,11458649,1,N,13.Danaus_plexippus_XM_061526465.1_Frame_1_rightoh_2_query_Exon_14,1,30,32

Vanessa_cardui,NC_061154.1,11447723,11447

min = set231_frame1, 0.102
5 top scores:
[['set231_frame1', 0.102], ['set109_frame2', 1.667], ['set9_frame0', 1.637], ['set155_frame2', 1.676], ['set197_frame2', 1.69]]
/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/8.Heliconiinae_Danainae/1.Blast_result/Vanessa_cardui/Period_gene_genomic_sequence_individual_exon_20.Vanessa_tameamea/Exon_24/for_alignment/alignment_Exon_24_translated_genomic_sequence_231_frame1.fa.txt
0 34
20.Vanessa_tameamea_XM_026643498.1_Frame_0_rightoh_0_query_Exon_24
VRVTSELIYKYQILTKDVAEVLSCDKKKLEKLEQ
0 34
set231_frame1
VRVTSELIYKYQILTKDMDEVLSCDKKKLEKLEQ
Original query = VRVTSELIYKYQILTKDVAEVLSCDKKKLEKLEQ
New query = VRVTSELIYKYQILTKDMDEVLSCDKKKLEKLEQ
Scaffold = NC_061154.1, start = 11468772, end = 11468905
Vanessa_cardui,NC_061154.1,11447072,11447212,1,Y,13.Danaus_plexippus_XM_061526465.1_Frame_0_rightoh_0_query_Exon_25,1,26,47

23
22
21
20
19
18
17
16
15
14
25
26
Vanessa_cardui,NC_061154.1,11458563,11458649,1,N,13.Danaus_plexippus_XM_061526

Scaffold = NW_020662887.1, start = 16071, end = 16207
Vanessa_tameamea,NW_020662887.1,21188,21391,0,N,13.Danaus_plexippus_XM_061526465.1_Frame_2_rightoh_1_query_Exon_8,1,67,67

6
5
4
3
2
1
10
Vanessa_tameamea,NW_020662887.1,16071,16207,0,N,13.Danaus_plexippus_XM_061526465.1_Frame_0_rightoh_2_query_Exon_1,1,29,29

Vanessa_tameamea,NW_020662887.1,22863,22974,0,N,13.Danaus_plexippus_XM_061526465.1_Frame_1_rightoh_0_query_Exon_10,1,37,37

Getting Gene
min = set137_frame1, 0.0
5 top scores:
[['set137_frame1', 0.0], ['set150_frame1', 1.774], ['set57_frame0', 1.807], ['set29_frame2', 1.814], ['set113_frame0', 1.791]]
/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/8.Heliconiinae_Danainae/1.Blast_result/Vanessa_tameamea/Period_gene_genomic_sequence_individual_exon_20.Vanessa_tameamea/Exon_8/for_alignment/alignment_Exon_8_translated_genomic_sequence_137_frame1.fa.txt
6 73
20.Vanessa_tameamea_XM_026643498.1_Frame_2_rightoh_1_query_Exon_8
PAQGNQTSTMVCRIRKYRGLTSGFRIKEKIVSYMPF

Scaffold = NW_020662887.1, start = 16071, end = 16207
Vanessa_tameamea,NW_020663288.1,3991791,3991928,0,Y,13.Danaus_plexippus_XM_061526465.1_Frame_1_rightoh_2_query_Exon_15,24,39,45

13
17
18
19
20
21
22
23
24
25
26
Vanessa_tameamea,NW_020662887.1,24793,24911,0,N,13.Danaus_plexippus_XM_061526465.1_Frame_0_rightoh_2_query_Exon_13,1,39,39

Vanessa_tameamea,NW_020662887.1,37469,37570,0,N,13.Danaus_plexippus_XM_061526465.1_Frame_0_rightoh_0_query_Exon_26,1,26,34

Getting Gene
min = set23_frame1, 0.0
5 top scores:
[['set23_frame1', 0.0], ['set94_frame2', 1.671], ['set275_frame2', 1.735], ['set84_frame1', 1.779], ['set204_frame2', 1.78]]
/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/8.Heliconiinae_Danainae/1.Blast_result/Vanessa_tameamea/Period_gene_genomic_sequence_individual_exon_20.Vanessa_tameamea/Exon_15/for_alignment/alignment_Exon_15_translated_genomic_sequence_23_frame1.fa.txt
42 91
20.Vanessa_tameamea_XM_026643498.1_Frame_1_rightoh_2_query_Exon_15
SKCALPVEREI

Scaffold = NW_020662887.1, start = 16071, end = 16207
Vanessa_tameamea,NW_020662887.1,33548,33706,0,Y,13.Danaus_plexippus_XM_061526465.1_Frame_0_rightoh_0_query_Exon_22,11,52,53

20
19
18
17
16
15
14
24
25
26
Vanessa_tameamea,NW_020662887.1,25551,25637,0,N,13.Danaus_plexippus_XM_061526465.1_Frame_1_rightoh_2_query_Exon_14,1,32,32

Vanessa_tameamea,NW_020662887.1,37469,37570,0,N,13.Danaus_plexippus_XM_061526465.1_Frame_0_rightoh_0_query_Exon_26,1,26,34

Getting Gene
min = set176_frame2, 0.0
5 top scores:
[['set176_frame2', 0.0], ['set252_frame1', 1.814], ['set36_frame2', 1.814], ['set81_frame2', 1.816], ['set249_frame2', 1.817]]
/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/8.Heliconiinae_Danainae/1.Blast_result/Vanessa_tameamea/Period_gene_genomic_sequence_individual_exon_20.Vanessa_tameamea/Exon_22/for_alignment/alignment_Exon_22_translated_genomic_sequence_176_frame2.fa.txt
0 61
20.Vanessa_tameamea_XM_026643498.1_Frame_0_rightoh_0_query_Exon_22
STEAIIENRNKRKLR